In [1]:
!python -m pip install torch torchvision torchaudio matplotlib pandas graphviz onnx onnxscript onnxruntime ndjson

  Using cached torch-2.5.1-cp311-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached torchvision-0.20.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.1 kB)
  Using cached torchaudio-2.5.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.4 kB)
  Using cached matplotlib-3.10.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached onnx-1.17.0-cp311-cp311-macosx_12_0_universal2.whl.metadata (16 kB)
  Using cached onnxscript-0.1.0-py3-none-any.whl.metadata (15 kB)
  Using cached onnxruntime-1.20.1-cp311-cp311-macosx_13_0_universal2.whl.metadata (4.5 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13

In [10]:
import ndjson
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os

In [11]:
import ndjson
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [12]:
# Regardons à quoi ressemble le dataset

with open("datasets/aircraft carrier.ndjson") as f:
    reader = ndjson.reader(f)
    print(reader.next())

{'word': 'aircraft carrier', 'countrycode': 'MY', 'timestamp': '2017-03-28 13:28:00.85173 UTC', 'recognized': True, 'key_id': '5475678961008640', 'drawing': [[[31, 32], [0, 7]], [[27, 37, 38, 35, 21], [4, 4, 6, 7, 3]], [[25, 28, 38, 39], [5, 10, 10, 7]], [[33, 34, 32], [4, 33, 44]], [[5, 188, 254, 251, 241, 185, 45, 9, 0], [50, 50, 54, 83, 86, 90, 86, 77, 52]], [[35, 35, 43, 125, 126], [85, 91, 92, 96, 90]], [[35, 76, 80, 77], [35, 37, 41, 47]], [[53, 50, 54, 80, 78], [34, 23, 22, 23, 34]]]}


In [26]:
# à la différence de MNIST où on avait un tuple (tensor, result), ici on devra convertir nous mêmes ces valeurs avec :
# result -> word ; tensor -> drawing

# On va utiliser le dataset Quick Draw de google, préparons une classe pour charger les fichiers ndjson
class QuickDrawDataset(Dataset):
    def __init__(self, root):
        self.data = []
        
        # on va tout chercher dans le dossier
        files = [f for f in os.listdir(root) if f.endswith('.ndjson')]
        print(f'{len(files)} files to load')
        loaded_files = 0
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, "r") as f:
                reader = ndjson.reader(f)
                for i, item in enumerate(reader):
                    self.data.append(self._convert_to_image(item["drawing"]))
            
            loaded_files += 1
            print(f'Progress: {loaded_files / len(files) * 100:.2f}%', end='\r')

        print(f'{len(files)} files loaded')

    def _convert_to_image(self, drawing):
        # Convertir les coordonnées en une image 28x28 pour retrouver mes reperes par rapport à MNIST
        canvas = np.zeros((28, 28), dtype=np.float32)
        for stroke in drawing:
            x_coords, y_coords = stroke
            for x, y in zip(x_coords, y_coords):
                x = int((x / 255) * 27)
                y = int((y / 255) * 27)
                canvas[y, x] = 1
        return canvas

    def __len__(self):
        return len(self.data)

    # on applique notre transform à la demande
    def __getitem__(self, idx):
        return ToTensor(self.data[idx])

In [27]:
# importer le dataset (vous pouvez vous faire un café)
train_dataset = QuickDrawDataset(
    root="datasets"
)
# addendum : ça fait long quand même
train_dataset

345 files to load


: 